In [2]:
import cv2
import numpy as np

## 모폴로지

모폴로지는 형태학이라는 뜻으로 영상 분야에서는 노이즈 제거, 구멍 메꾸기, 연결되지 않은 경계 이어붙이기 등 형태학적 관점에서의 영상 연산을 말합니다.

모폴로지는 주로 형태를 다루는 연산이므로 바이너리 이미지를 대상으로 합니다
대표적인 연산은 침식과 팽창이며, 이 둘을 결합한 열리마과 닫힘 연산이 있음

In [12]:
## 침식 연산

img = cv2.imread("./img/morph_dot.png")

k = cv2.getStructuringElement(cv2.MORPH_RECT, (4,4))
erosion_morph_rect=cv2.erode(img, k) #침식 연산 작용

k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
erosion_morph_ellipse=cv2.erode(img, k) #침식 연산 작용

k = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
erosion_morph_cross=cv2.erode(img, k) #침식 연산 작용

merge = np.hstack((img, erosion_morph_rect, erosion_morph_ellipse, erosion_morph_cross))
cv2.imshow("Erode", merge)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
## 팽창 연산 (1로 채워진 영역이 온전히 덮이지 않으면 1로 채워넣고..) 1로 채워넣는다가...흰색으로 채운다..

img = cv2.imread("./img/morph_hole.png")


k = cv2.getStructuringElement(cv2.MORPH_RECT, (4,4))
erosion_morph_rect=cv2.dilate(img, k) #팽창연산

k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
erosion_morph_ellipse=cv2.dilate(img, k) 

k = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
erosion_morph_cross=cv2.dilate(img, k) 


merge = np.hstack((img, erosion_morph_rect, erosion_morph_ellipse, erosion_morph_cross))
cv2.imshow("Dilation", merge)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
## 열림 닫힘...

img1 = cv2.imread("./img/morph_dot.png", cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread("./img/morph_hole.png", cv2.IMREAD_GRAYSCALE)

k = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
opening = cv2.morphologyEx(img1, cv2.MORPH_OPEN, k)
closing  = cv2.morphologyEx(img2, cv2.MORPH_CLOSE, k)

merge1 = np.hstack((img1, opening))
merge2 = np.hstack((img2, closing))
merge3= np.vstack((merge1, merge2))
cv2.imshow("merge", merge3)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [19]:
## 모폴로지 그라디언트(경계값만..출력) = 경계검출

img = cv2.imread("./img/morphological.png")

k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
gradient=cv2.morphologyEx(img, cv2.MORPH_GRADIENT, k)
merged = np.hstack((img,gradient))

cv2.imshow("merge", merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
## 모폴로지 탑햇(원본- 열림) :밝기 값이 크게 튀는 영역을 강조할 수 있음
## 모폴로지 블랙햇(닫힘 - 열림) : 어두운 부분을 강조함

img = cv2.imread("./img/moon_gray.jpg")

k = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5)) #커널 생성..

tophat = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, k) #모폴로지 탑햇
blackhat = cv2.morphologyEx(img, cv2.MORPH_BLACKHAT,k) #모폴로지 블랙햇

merge = np.hstack((img, tophat, blackhat))
cv2.imshow("merge", merge)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 이미지 피라미드

영상의 크기를 단계적으로 축소 또는 확대하여 피라미드처럼 쌓아 놓는 것을 말함

In [28]:
img = cv2.imread("./img/girl.jpg")

smaller=cv2.pyrDown(img) # 가우시안 이미지 피라미드 축소
bigger = cv2.pyrUp(img)
bigger_small=cv2.pyrDown(bigger) #이미지 영상을 크게했다가..다시 작게해서 원본 영상으로 돌아오는 것이 불가능함

cv2.imshow("img", img)
cv2.imshow("smaller", smaller)
cv2.imshow("bigger", bigger)
cv2.imshow("bigger_small", bigger_small)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [29]:
## 라플라이사인 피라미드로 영상 복원하기(다시 작게해서 원본 영상으로 돌아오는 것이 불가능하기때문에..)

img = cv2.imread("./img/taekwonv1.jpg")

smaller = cv2.pyrDown(img)
bigger = cv2.pyrUp(smaller) #축소한 영상을 가우시안 피라미드로 확대함

#원본에서 확대한 영상을 빼기(라플라시안)
laplacian = cv2.subtract(img, bigger)
## 확대한 영상에 라플라시안 영상을 더해서 복원함
restored = bigger + laplacian

merged = np.hstack((img, laplacian, bigger, restored))
cv2.imshow("merged", merged)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 실전 워크숍(모자이크 처리2)

In [35]:
ksize = 30
win_title = "mosaic"
img = cv2.imread("./img/taekwonv1.jpg")

while True:
    x,y,w,h = cv2.selectROI(win_title, img, False)
    if w>0 and h>0:
        img1=img.copy()
        roi = img1[y:y+h, x:x+w]
        roi=cv2.blur(roi, (ksize, ksize)) #ksize사이즈만큼 블러처리함
        img1[y:y+h, x:x+w] = roi
        merged= np.hstack((img, img1))
        cv2.imshow("merge", merged)
    else: 
        break
        
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
## 스케치 효과 카메라
import cv2
import numpy as np
cap = cv2.VideoCapture(0)
while cap.isOpened():
    rat, frame=cap.read() #프레임 읽기
    frame=cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    if cv2.waitKey(1) ==27:
        break
    img_gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    img_gray=cv2.GaussianBlur(img_gray, (9,9), 0) #잡음제거를 위한 가우시안 필터
    edges=cv2.Laplacian(img_gray, -1, None, 5) # 라플레시안 필터를 엣지만 검출(경계선 검출)
    
    ret, sketch=cv2.threshold(edges, 70,255, cv2.THRESH_BINARY_INV)
    
    #경계선 강조를 위한 팽창 연산
    kernel=cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
    sketch1=cv2.erode(sketch, kernel)
    
    sketch1 = cv2.medianBlur(sketch1,5) #자연스럽게 하기위한 블러 필터 적용
    img_sketch=cv2.cvtColor(sketch1, cv2.COLOR_GRAY2BGR)
    
    # 컬러 이미지 선명선을 없애기 윟ㄴ 평균 블러 필터 적용
    img_paint=cv2.blur(frame, (10,10))
    img_paint = cv2.bitwise_and(frame, img_sketch) #컬러영상과 스케치영상을 합성함.
    
    merged = np.hstack((frame, img_sketch, img_paint ))
    cv2.imshow("img_sketch",merged)
    #cv2.imshow("sketch1", sketch)
    #cv2.imshow("edge", edges) #경계선만 검출함

    
cap.release()
cv2.destroyAllWindows()

In [21]:
import cv2
import numpy as np
cap = cv2.VideoCapture(0)
while cap.isOpened():
    rat, frame=cap.read() #프레임 읽기
    frame=cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    if cv2.waitKey(1) ==27:
        break
    #img_sketch, color=cv2.pencilSketch(frame)
    #img_sketch=cv2.stylization(frame)
    #img_sketch = cv2.detailEnhance(frame)
    img_sketch=cv2.edgePreservingFilter(frame)
    
    #merged = np.hstack((frame, img_sketch))
    cv2.imshow("img_sketch",img_sketch)
cap.release()
cv2.destroyAllWindows()